In [23]:
import pandas as pd

excel_file = pd.ExcelFile('sampledata_1.xlsx')
df = excel_file.parse('hardpaste')

required_columns = ['company', 'gender']
if not all(col in df.columns for col in required_columns):
    print("The sheet must contain 'Company' and 'Gender' columns.")
    exit()
df


,attage,issue,planner,policy,gender,birthdate,dod,clm_value,amountband,notification,regioncode,company
0,0-19,2005-09-24,T,PPL113,F,1947-03-17,2007-09-09,95249,A2,2010-01-09,2,Che
1,20-29,2002-09-23,Y,PPL123,M,1959-11-16,2005-07-05,167152,A3,2005-07-05,1,Che
2,30-39,1999-09-24,T,PPL487,M,1955-05-12,2004-01-25,119201,A1,2004-03-25,1,Che
3,0-19,2000-11-18,T,PPL668,M,1965-04-05,2005-06-14,120075,A4,2005-11-14,3,Che
4,60-69,1999-09-02,Y,PPL149,M,1920-01-12,2003-06-03,90272,A5,2005-06-03,2,Che
...,...,...,...,...,...,...,...,...,...,...,...,...
79,40-49,1998-12-22,T,PPL675,M,1939-05-13,2001-10-12,97412,A2,2002-03-12,2,Ind
80,60-69,2014-06-19,T,PPL085,M,1994-06-26,2019-02-25,181562,A2,2021-06-25,1,Ind
81,60-69,2001-04-16,T,PPL118,F,1957-02-24,2005-10-28,129773,A3,2008-02-28,3,Ind
82,60-69,2001-04-16,T,PPL006,F,1957-04-05,2005-12-12,97508,A3,2007-12-12,1,Ind


In [24]:
# Get unique values
companies = df['company'].dropna().unique()
genders = df['gender'].dropna().unique()

# Show options and get valid input from user
def get_valid_index(prompt, options):
    while True:
        print(f"\nSelect a {prompt}:")
        for i, option in enumerate(options, start=1):
            print(f"{i}. {option}")
        try:
            choice = int(input("Enter the number for your choice: "))
            if 1 <= choice <= len(options):
                return choice - 1
            else:
                print(f"Please enter a number between 1 and {len(options)}.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Get valid choices
company_choice = get_valid_index("Company", companies)
gender_choice = get_valid_index("Gender", genders)

# Get selected values
selected_company = companies[company_choice]
selected_gender = genders[gender_choice]

# Filter the DataFrame
df = df[(df['company'] == selected_company) & (df['gender'] == selected_gender)]

# Display the filtered DataFrame
print(f"\nFiltered data for Company: {selected_company} and Gender: {selected_gender}")
df


Select a Company:
1. Che
2. Ind


Enter the number for your choice:  1



Select a Gender:
1. F
2. M


Enter the number for your choice:  2



Filtered data for Company: Che and Gender: M


,attage,issue,planner,policy,gender,birthdate,dod,clm_value,amountband,notification,regioncode,company
1,20-29,2002-09-23,Y,PPL123,M,1959-11-16,2005-07-05,167152,A3,2005-07-05,1,Che
2,30-39,1999-09-24,T,PPL487,M,1955-05-12,2004-01-25,119201,A1,2004-03-25,1,Che
3,0-19,2000-11-18,T,PPL668,M,1965-04-05,2005-06-14,120075,A4,2005-11-14,3,Che
4,60-69,1999-09-02,Y,PPL149,M,1920-01-12,2003-06-03,90272,A5,2005-06-03,2,Che
5,70-79,2014-01-28,Y,PPL842,M,1937-02-08,2016-03-14,192966,A1,2018-04-14,1,Che
6,80-UP,1988-12-21,Y,PPL713,M,1944-10-26,1990-10-07,134554,A4,1992-09-07,2,Che
7,0-19,1999-12-24,Y,PPL104,M,1953-10-03,2002-11-13,81024,A4,2004-10-13,2,Che
8,20-29,2000-07-24,Y,PPL925,M,1940-05-04,2002-03-28,89684,A2,2004-07-28,2,Che
9,30-39,1999-12-28,Y,PPL846,M,1960-02-20,2002-11-03,196132,A4,2004-10-03,1,Che
10,0-19,1999-02-09,Y,PPL825,M,1963-02-09,2000-07-03,191645,A3,2001-03-03,3,Che


In [26]:
# Make a copy of the DataFrame to avoid SettingWithCopyWarning
df_copy = df.copy()

# Ensure 'dod' and 'notification' columns are datetime type
df_copy['dod'] = pd.to_datetime(df_copy['dod'])
df_copy['notification'] = pd.to_datetime(df_copy['notification'])

# Group by 'dod' and 'notification' and sum 'clm_value'
summary_df = (
    df_copy.groupby(['dod', 'notification'], as_index=False)
           .agg(sum_clm_value=('clm_value', 'sum'))
)

# Calculate lag in months (floor the value)
def diff_in_months(end_date, start_date):
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

summary_df['lag'] = summary_df.apply(lambda row: diff_in_months(row['notification'], row['dod']), axis=1)

# Extract year and month from 'dod'
summary_df['dod_year'] = summary_df['dod'].dt.year
summary_df['dod_month'] = summary_df['dod'].dt.month

summary_df.head(10)

,dod,notification,sum_clm_value,lag,dod_year,dod_month
0,1990-10-07,1992-09-07,134554,23,1990,10
1,2000-07-03,2001-03-03,191645,8,2000,7
2,2002-02-15,2004-06-15,123967,28,2002,2
3,2002-03-28,2004-07-28,89684,28,2002,3
4,2002-09-05,2005-02-05,126316,29,2002,9
5,2002-09-15,2004-09-15,108137,24,2002,9
6,2002-11-03,2004-10-03,196132,23,2002,11
7,2002-11-13,2004-10-13,81024,23,2002,11
8,2003-06-03,2005-06-03,90272,24,2003,6
9,2004-01-25,2004-03-25,119201,2,2004,1
